In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch
from datasets import load_dataset


model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Dataset

In [ ]:
CUTOFF_LEN = 256

dataset = load_dataset("harpreetsahota/modern-to-shakesperean-translation") 
print("dataset", dataset)
train_data = dataset["train"] 

def generate_prompt(user_query):
    sys_msg= "Translate the given text to Shakespearean style."
    p =  "<s> [INST]" + sys_msg +"\n"+ user_query["modern"] + "[/INST]" +  user_query["shakespearean"] + "</s>"
    return p 

def tokenize(prompt):
    return tokenizer(
        prompt + tokenizer.eos_token,
        truncation=True,
        max_length=CUTOFF_LEN ,
    )

train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)), remove_columns=["modern" , "shakespearean"])

## Set up the model and tokenizer

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

## Set up PEFT (Parameter-Efficient Fine-Tuning)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

## Set up the training arguments

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="test-finetuning",
    max_steps=10, 
    per_device_train_batch_size=1,
    learning_rate=1e-5,
    save_steps=100000,
    optim="sgd",
    optim_target_modules=["attn", "mlp"],
)

## Initialize the trainer and fine-tune the model

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
)

trainer.train()

## Merge the adapter and model back together

In [ ]:
adapter_model = trainer.model
merged_model = adapter_model.merge_and_unload()

In [ ]:
# Save the model and its configuration
model_save_path = "finetuned_model"

merged_model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load the model and tokenizer
model = AutoModel.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

# Inference
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model(**inputs)
outputs